# YAML Joiner

## Import module



In [1]:
# Import the modules

import os

In [14]:
def import_file(file):
    with open(file) as f:
        lines = f.readlines()
        
    dependencies = []
    pip = []
    write_dep = False
    write_pip = False
    
    for line in lines:
        if write_dep:
            dependencies.append(line)
        if write_pip:
            pip.append(line)
            
        if "dependencies" in line:
            write_dep = True
        
        if "pip:" in line:
            write_pip = True
            write_dep = False
            
        
        if "prefix:" in line:
            break
    
    dependencies = dependencies[:-1]
    pip = pip[:-1]
    return dependencies, pip

In [15]:
slave_dep, slave_pip = import_file("src/PDFv1.yaml")
master_dep, master_pip = import_file("src/proto_bal.yaml")

In [16]:
def add_master(masters, slaves):
    
    output = []
    for slave in slaves:
        
        slave_name = slave[:slave.index("=")]
        counter = 0
        
        for master in masters:
            if slave_name in master:
                counter +=1
                break
        
        if counter == 0:
            output.append(slave)
            
    return output

In [17]:
dep_add = add_master(master_dep, slave_dep)

master_dep += dep_add

In [18]:
pip_add = add_master(master_pip, slave_pip)

master_pip += pip_add

In [19]:
slave_pip

['  - argcomplete==1.8.2\n',
 '  - backports-abc==0.5\n',
 '  - backports.shutil-get-terminal-size==1.0.0\n',
 '  - backports.ssl-match-hostname==3.5.0.1\n',
 '  - beautifulsoup4==4.5.3\n',
 '  - chardet==2.3.0\n',
 '  - distribute==0.7.3\n',
 '  - docx2txt==0.6\n',
 '  - ebooklib==0.15\n',
 '  - ipython-genutils==0.2.0\n',
 '  - jupyter-client==5.1.0\n',
 '  - jupyter-console==5.2.0\n',
 '  - jupyter-core==4.3.0\n',
 '  - lxml==4.1.0\n',
 '  - olefile==0.44\n',
 '  - pdfminer==20140328\n',
 '  - pillow==4.3.0\n',
 '  - pocketsphinx==0.1.3\n',
 '  - prompt-toolkit==1.0.15\n',
 '  - pypdf2==1.26.0\n',
 '  - python-pptx==0.6.5\n',
 '  - slate==0.3\n',
 '  - speechrecognition==3.6.3\n',
 '  - textract==1.6.1\n',
 '  - win-unicode-console==0.5\n',
 '  - xlrd==1.0.0\n',
 '  - xlsxwriter==1.0.2\n']

In [13]:
master_pip

['  - backports-abc==0.5\n',
 '  - backports.shutil-get-terminal-size==1.0.0\n',
 '  - backports.ssl-match-hostname==3.5.0.1\n',
 '  - chardet==3.0.4\n',
 '  - et-xmlfile==1.0.1\n',
 '  - forex-python==0.3.2\n',
 '  - ipython-genutils==0.2.0\n',
 '  - jupyter-client==5.1.0\n',
 '  - jupyter-console==5.2.0\n',
 '  - jupyter-core==4.3.0\n',
 '  - prompt-toolkit==1.0.15\n',
 '  - simplejson==3.11.1\n',
 '  - urllib3==1.22\n',
 '  - win-unicode-console==0.5\n',
 '  - argcomplete==1.8.2\n',
 '  - beautifulsoup4==4.5.3\n',
 '  - distribute==0.7.3\n',
 '  - docx2txt==0.6\n',
 '  - ebooklib==0.15\n',
 '  - lxml==4.1.0\n',
 '  - olefile==0.44\n',
 '  - pdfminer==20140328\n',
 '  - pillow==4.3.0\n',
 '  - pocketsphinx==0.1.3\n',
 '  - pypdf2==1.26.0\n',
 '  - python-pptx==0.6.5\n',
 '  - slate==0.3\n',
 '  - speechrecognition==3.6.3\n',
 '  - textract==1.6.1\n',
 '  - xlrd==1.0.0\n',
 '  - xlsxwriter==1.0.2\n']

In [50]:
def import_headers(file):
    
    with open(file) as f:
        lines = f.readlines()
        
    header = []

    for line in lines:
        header.append(line)
        
        if "dependencies" in line:
            break
    
    closing = lines[-2:-1]
    
    return header, closing

In [51]:
master_head, master_closing = import_headers('src/proto_bal.yaml')

In [52]:
master_closing

['prefix: C:\\ProgramData\\Anaconda2\\envs\\proto_bal\n']

In [53]:
new_yaml = master_head + master_dep + ['  - pip:\n'] + master_pip + master_closing

In [54]:
def write_to_file(filename, list_lines, path = 'dir/'):
    with open(path + filename, "w") as f:
        for line in list_lines:
            f.write(line)
            
    return

In [55]:
write_to_file('proto_bal.yaml', new_yaml)

In [56]:
new_yaml

['name: proto_bal\n',
 'channels:\n',
 '- conda-forge\n',
 '- defaults\n',
 'dependencies:\n',
 '- ca-certificates=2017.11.5=0\n',
 '- asn1crypto=0.22.0=py27_0\n',
 '- backports=1.0=py27_0\n',
 '- backports_abc=0.5=py27_0\n',
 '- bleach=1.5.0=py27_0\n',
 '- certifi=2016.2.28=py27_0\n',
 '- cffi=1.10.0=py27_0\n',
 '- colorama=0.3.9=py27_0\n',
 '- configparser=3.5.0=py27_0\n',
 '- cryptography=1.8.1=py27_0\n',
 '- cycler=0.10.0=py27_0\n',
 '- decorator=4.1.2=py27_0\n',
 '- entrypoints=0.2.3=py27_0\n',
 '- enum34=1.1.6=py27_0\n',
 '- et_xmlfile=1.0.1=py27_0\n',
 '- functools32=3.2.3.2=py27_0\n',
 '- get_terminal_size=1.0.0=py27_0\n',
 '- html5lib=0.999=py27_0\n',
 '- icu=57.1=vc9_0\n',
 '- idna=2.6=py27_0\n',
 '- inflection=0.3.1=py27_0\n',
 '- ipaddress=1.0.18=py27_0\n',
 '- ipykernel=4.6.1=py27_0\n',
 '- ipython=5.3.0=py27_0\n',
 '- ipython_genutils=0.2.0=py27_0\n',
 '- ipywidgets=6.0.0=py27_0\n',
 '- jdcal=1.3=py27_0\n',
 '- jinja2=2.9.6=py27_0\n',
 '- jpeg=9b=vc9_0\n',
 '- jsonschema=